In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
#Importing all the necessary libraries
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision.utils import save_image


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
os.makedirs("models", exist_ok=True)
os.makedirs("samples", exist_ok=True)

In [4]:
#Setting the hyperparameters
latent_dim = 100
batch_size = 128
epochs = 50
lr = 0.0002

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

dataset = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform
)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


100%|██████████| 9.91M/9.91M [00:11<00:00, 888kB/s] 
100%|██████████| 28.9k/28.9k [00:00<00:00, 93.7kB/s]
100%|██████████| 1.65M/1.65M [00:02<00:00, 700kB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 2.32MB/s]


In [6]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), 1, 28, 28)


In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        img = img.view(img.size(0), -1)
        return self.model(img)


In [8]:
G = Generator().to(device)
D = Discriminator().to(device)

criterion = nn.BCELoss()

optimizer_G = optim.Adam(G.parameters(), lr=lr)
optimizer_D = optim.Adam(D.parameters(), lr=lr)


In [9]:
for epoch in range(epochs):
    for i, (real_imgs, _) in enumerate(dataloader):

        real_imgs = real_imgs.to(device)
        batch_size = real_imgs.size(0)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ------------------
        # Train Discriminator
        # ------------------
        optimizer_D.zero_grad()

        real_loss = criterion(D(real_imgs), real_labels)

        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = G(z).detach()
        fake_loss = criterion(D(fake_imgs), fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # ------------------
        # Train Generator
        # ------------------
        optimizer_G.zero_grad()

        z = torch.randn(batch_size, latent_dim).to(device)
        fake_imgs = G(z)
        g_loss = criterion(D(fake_imgs), real_labels)

        g_loss.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{epochs}] | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

    # Save sample images
    save_image(fake_imgs[:25], f"samples/epoch_{epoch+1}.png", nrow=5, normalize=True)


Epoch [1/50] | D Loss: 0.5130 | G Loss: 5.1318
Epoch [2/50] | D Loss: 0.4000 | G Loss: 7.1363
Epoch [3/50] | D Loss: 2.9253 | G Loss: 1.4796
Epoch [4/50] | D Loss: 0.8252 | G Loss: 3.5495
Epoch [5/50] | D Loss: 1.1916 | G Loss: 1.1332
Epoch [6/50] | D Loss: 2.2440 | G Loss: 0.8486
Epoch [7/50] | D Loss: 0.8353 | G Loss: 2.0850
Epoch [8/50] | D Loss: 1.7532 | G Loss: 1.2725
Epoch [9/50] | D Loss: 0.8866 | G Loss: 1.2920
Epoch [10/50] | D Loss: 0.6563 | G Loss: 1.3616
Epoch [11/50] | D Loss: 1.3050 | G Loss: 1.0982
Epoch [12/50] | D Loss: 1.7889 | G Loss: 1.7664
Epoch [13/50] | D Loss: 0.8600 | G Loss: 1.7893
Epoch [14/50] | D Loss: 0.8330 | G Loss: 1.8321
Epoch [15/50] | D Loss: 0.6381 | G Loss: 2.0012
Epoch [16/50] | D Loss: 0.1986 | G Loss: 4.2306
Epoch [17/50] | D Loss: 0.3249 | G Loss: 3.0636
Epoch [18/50] | D Loss: 0.6204 | G Loss: 4.0554
Epoch [19/50] | D Loss: 0.0418 | G Loss: 4.0470
Epoch [20/50] | D Loss: 0.0177 | G Loss: 5.3680
Epoch [21/50] | D Loss: 0.0321 | G Loss: 4.9575
E

In [11]:
torch.save(G.state_dict(), "models/generator.pth")
torch.save(D.state_dict(), "models/discriminator.pth")


#Now using the trained model to generate images

In [12]:
import torch
import torch.nn as nn
from torchvision.utils import save_image
import os


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
latent_dim = 100
os.makedirs("generated", exist_ok=True)


In [15]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 28 * 28),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        return img.view(z.size(0), 1, 28, 28)


In [16]:
G = Generator().to(device)
G.load_state_dict(torch.load("models/generator.pth", map_location=device))
G.eval()


Generator(
  (model): Sequential(
    (0): Linear(in_features=100, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1024, bias=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=784, bias=True)
    (7): Tanh()
  )
)

In [21]:
with torch.no_grad():
    z = torch.randn(64, latent_dim).to(device)
    fake_images = G(z)
    save_image(fake_images, "generated/generated_digit5.png", nrow=5, normalize=True)
